# Retrieval-Augmented Generation (RAG)

We experimented with LLMs as well as with retrieval models. It's time to combine the two.

In [1]:
import os

import bm25s
import pandas as pd
import Stemmer
from datasets import Dataset
from dotenv import load_dotenv
from openai import AzureOpenAI
from sentence_transformers import SentenceTransformer


In [2]:
# Load environment variables from .env file
load_dotenv()

llm_endpoint = "https://aa-dsa-training-msca.openai.azure.com/"

llm_model_name = "gpt-5-nano"
llm_deploy_name = "gpt-5-nano"
#llm_model_name = "gpt-5-mini"
#llm_deploy_name = "gpt-5-mini"
#llm_model_name = "gpt-5"
#llm_deploy_name = "gpt-5"

subscription_api_key = os.getenv("OPENAI_API_KEY")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_key=subscription_api_key,
    api_version=api_version,
    azure_endpoint=llm_endpoint,
)

In the first example, we were summarizing fixed text:

In [3]:
text = """
Just as people start bringing out their shorts for the start of summer, 
one Texas town had to reach for something rarely seen in late May: a snowplough.
Parts of the state saw a dramatic 50F temperature drop on Wednesday thanks to 
a giant dump of hail, some “DVD-sized”. The storm made western Texas look, 
briefly, like a winter wonderland.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

response = client.chat.completions.create(model=llm_deploy_name,
                                          messages=[
                                              {"role": "system", "content": "You are a helpful assistant that summarizes text."},
                                              {"role": "user", "content": prompt}
                                          ],
                                          max_completion_tokens=800,
                                          )

print(response.choices[0].message.content)

As summer arrives, a Texas town briefly used a snowplough after a hailstorm dumped DVD-sized hail and caused a 50-degree drop in temperature, leaving western Texas looking like a winter wonderland.


Let's change this now to add the retriever.  
Instead of a fixed text to summarize, we will augment the prompt, the query, automatically and dynamically with the retrieved text.

In [4]:
financial_retrieval_corpus = pd.read_parquet("../../data/financial_retrieval_corpus.parquet")
financial_retrieval_queries = pd.read_parquet("../../data/financial_retrieval_queries.parquet")

In [5]:
stemmer = Stemmer.Stemmer("english")

# Tokenize the corpus and only keep the ids (faster and saves memory)
corpus_tokens = bm25s.tokenize(financial_retrieval_corpus['content'], stopwords="en", stemmer=stemmer)

# Create the BM25 model and index the corpus
retriever = bm25s.BM25()
retriever.index(corpus_tokens)

Split strings:   0%|          | 0/25 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/25 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/25 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/25 [00:00<?, ?it/s]

`financial_retrieval_queries` contains about 100 queries and their expected answer.

In [9]:
financial_retrieval_queries.head(5)

,query,answer,content_index
0,What was the increase in revenue from the prev...,Up 101%,0
1,What was the revenue in second quarter?,$13.51 billion,0
2,How many shares were repurchased in second qua...,7.5 million shares,0
3,Who is the CEO of Nvidia?,Jensen Huang,0
4,What was the percentage increase in data cente...,Up 141% from Q1,0


In [31]:
query = bm25s.tokenize(financial_retrieval_queries["query"][0], stemmer=stemmer)
retrieved_ids, retrieved_scores = retriever.retrieve(query, k=3)
retrieved_documents = [financial_retrieval_corpus['content'][id] for id, score in zip(retrieved_ids[0], retrieved_scores[0]) if score > 0]

response = client.chat.completions.create(model=llm_deploy_name,
                                          messages=[
                                              {"role": "system", "content": "You are a helpful assistant that answers questions by summarizing the given content. Answer using ONLY using the provided document sources. If the answer isn't there, say \"I don't know.\" Do not use prior knowledge"},
                                              {"role": "user", "content": f"Based on the retrieved context: {"\n\n\n".join(retrieved_documents)}, answer the question {query} step by step, first identifying key facts, then reasoning through the answer."}
                                          ],
                                          max_completion_tokens=9800,
                                          )

print(response.choices[0].message.content)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Step 1 – Key facts from the document:
- NVIDIA’s second quarter revenue was $13.51 billion.
- It was up 101% from a year ago (i.e., year-over-year increase).

Step 2 – Reasoning:
- The phrase “revenue increase from the previous year” corresponds to the year-over-year percentage change reported in the document.

Step 3 – Answer:
- The revenue increased by 101% from the previous year.
